Gapminder dashboard
Use the gapminder dataset from plotly.express and create a dashboard. Do explorative data analysis, pick out graphs and metrics you are interested in displaying, then draw a wireframe by hand or a simple tool such as paint, powerpoint or figma or something else. Use this wireframe to guide your design of your dashboard.

In [7]:
import plotly.express as px
import pandas as pd
import duckdb 

df = px.data.gapminder()
df.shape

(1704, 8)

In [4]:
df.columns

Index(['country', 'continent', 'year', 'lifeExp', 'pop', 'gdpPercap',
       'iso_alpha', 'iso_num'],
      dtype='object')

In [5]:
df.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [117]:
df_continent = duckdb.query("""--sql
             SELECT continent, year, avg(lifeExp) as avg_life_exp, avg(gdpPercap) as avg_gdp_person
            FROM df
             GROUP BY continent, year
                            ORDER BY year, continent
             """).df()
df_continent

,continent,year,avg_life_exp,avg_gdp_person
0,Africa,1952,39.135500,1252.572466
1,Americas,1952,53.279840,4079.062552
2,Asia,1952,46.314394,5195.484004
3,Europe,1952,64.408500,5661.057435
4,Oceania,1952,69.255000,10298.085650
5,Africa,1957,41.266346,1385.236062
6,Americas,1957,55.960280,4616.043733
7,Asia,1957,49.318544,5787.732940
8,Europe,1957,66.703067,6963.012816
9,Oceania,1957,70.295000,11598.522455


In [118]:
fig = px.line(df_continent, 
              x= "year", 
              y="avg_life_exp", 
              color="continent", 
              title="Yearly average life expectancy per continent",
              labels=dict(continent= "Continent", avg_life_exp = "Avg life expectancy", year = "Year")
              )

fig.update_layout(plot_bgcolor="white", hovermode="x unified")

fig.update_xaxes(
    showspikes=True,
    spikemode = "across",
    spikesnap = "cursor",
    )

fig.show()


In [119]:
fig = px.line(df_continent, 
              x= "year", 
              y="avg_gdp_person", 
              color="continent", 
              title="Yearly average GDP per person per continent",
              labels=dict(continent= "Continent", avg_life_exp = "Avg life expectancy", year = "Year")
              )

fig.update_layout(plot_bgcolor="white", hovermode="x unified")

fig.update_xaxes(
    showspikes=True,
    spikemode = "across",
    spikesnap = "cursor",
    )

fig.show()


In [95]:
#population bar chart with year filters
pop_df = df[["continent", "year", "pop"]]
filtered_pop_df = pop_df[pop_df['year'] == 2007].groupby(["continent"]).sum("pop").reset_index()
filtered_pop_df

,continent,year,pop
0,Africa,104364,929539692
1,Americas,50175,898871184
2,Asia,66231,3811953827
3,Europe,60210,586098529
4,Oceania,4014,24549947


In [105]:
def bar_year(df, year):
    filtered_pop_df = df[df['year'] == year].groupby(["continent"]).sum("pop").reset_index()

    fig = px.bar(filtered_pop_df,
                 x= "continent",
                 y= "pop",
                 color= "continent",
                 title= f"Population by continent in {year}",
                 labels=dict(pop="Population", continent="Continent")
                 )

    fig.show()

bar_year(pop_df, 1967)

In [131]:
df1 = px.data.gapminder().query("year==2007")
df1.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32


In [145]:
def map_life_exp(df, continent="europe"):
    fig = px.choropleth(df, locations="iso_alpha",
                        color="lifeExp",
                        scope=continent,
                        hover_name="country",
                        )
    fig.show()

map_life_exp(df1, "asia")

In [144]:
df["year"].unique()

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007])